In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import pyarrow

# Suspicion Score

There are three reasons why a node might be suspicious: 
1. The KYC data (node data, eg. occupation) is suspicious
2. The transactions (edges, e.g. e-transfer messages) around that node are suspicious
3. The graph structure around that node is suspicious (e.g. in a 'mule' structure)

The purpose of this notebook is to aggregate those three factors (and the indicators contained therein) to come up with an explainable total node suspicion score. 

TODO:

- ~Aggregate node indicators (JWB)~
- Aggregate e-transfer features (MM)
- Aggregate wire-transfer features (JWB)
- ~Aggregate cash-transfer features (JWB)~
- Aggregate general features (AGG)
- PageRank (MM)
- Find a suspicious network (AGG)

# Node Suspicion Score

In [28]:
DATAPATH = Path('../data/processed')
KYCPATH = DATAPATH / 'kyc.parquet'
CASHPATH = DATAPATH / 'cash.parquet'

## KYC Aggregation
We first combine the KYC features `label`, `occ_int`, `occ_wealth`, `occ_animal` into one one KYC score. To do this, we assume that the occupation flags are additive, and then take the max of the occupation aggregate score with the `label` feature. 

Basically: If they have been flagged for money laundering, their node should have the heighest suspision score, otherwise we take a weighted average of the occupation flags.

`kyc_agg`$=\max$(`label`, $\texttt{avg}$(`occ_int`, `occ_wealth`, `occ_animal`))


In [34]:
#Optional feature weights
W_OCC_INT = 1/3
W_OCC_WEALTH = 1/3
W_OCC_ANIMAL = 1/3

In [38]:
kyc_df = pd.read_parquet(KYCPATH)

def kyc_agg(r): 
    occ_agg = W_OCC_INT*r.occ_int + W_OCC_WEALTH*r.occ_wealth + W_OCC_ANIMAL*r.occ_animal
    return max(r.label, occ_agg)

kyc_df['kyc_agg'] = kyc_df.apply(lambda r: kyc_agg(r), axis=1)
kyc_df.sample(3)

,name,gender,occupation,age,tenure,cust_id,occ_wealth,occ_animal,occ_int,label,kyc_agg
21989,AMY JOHNSON,female,Pet Groomer,33.0,8.0,CUST68389688,0,1,0,0,0.333333
62717,JOHN WATKINS,male,Gardener,34.0,3.0,CUST72834415,0,0,0,0,0.000000
68089,DR.KELLY CARTER,female,Food Critic,33.0,7.0,CUST80817667,0,0,0,0,0.000000


## Cash Aggregation
For the purposes of this project, we consider cash transactions as only occuring between one customer (i.e. the bank does *not* count as a customer). This prevents graph neighbourhoods from being too dense (everyone is connected to the bank, so everyone is a 2-hop neighbour). 

For practical purposes, this means that indicators related to deposit/withdrawal size become node scores, rather than edge scores.

There is only one cash feature currently, `c_large`. `cash_agg` is therefore defined as the proportion of a customers cash transactions that have been defined as being unusually large.

In [54]:
cash_df = pd.read_parquet(CASHPATH)
cash_agg_df = cash_df.groupby('cust_id')['c_large'].mean().reset_index()
cash_agg_df = cash_agg_df.rename(columns={'c_large':'c_agg'})
cash_agg_df.sample(3)

,cust_id,c_agg
13739,CUST30370806,0.0
60855,CUST99887405,0.0
26105,CUST48967972,0.0


## Node Score Calculation
We assume that the total node suspicion score is an average of the aggregate kyc suspicion score  and the aggregate cash suspicion score above.

In [62]:
#Join KYC and Cash Data
merged = kyc_df.merge(cash_agg_df, on='cust_id', how='left')
merged['c_agg'] = merged['c_agg'].fillna(0)

#Aggregate 
node_df = merged[['cust_id', 'kyc_agg', 'c_agg']].copy()
node_df['score'] = (node_df['kyc_agg'] + node_df['c_agg'])/2

#Clean
node_df = node_df[['cust_id', 'score']]


#Export
node_df.to_parquet(DATAPATH/'node_score.parquet', index=False)
node_df.sort_values('score', ascending=False)

,cust_id,score
150089,CUST87731042,1.0
165103,CUST51289421,1.0
110190,CUST21142684,1.0
180831,CUST55502501,1.0
57930,CUST27210892,1.0
...,...,...
73441,CUST87917518,0.0
73442,CUST10383819,0.0
73443,CUST81898171,0.0
73444,CUST48514678,0.0


# Edge Suspicion Score
Features starting with **agg** are aggregates of pre-computed transaction features.

In [63]:
DATAPATH = Path('../data/processed')
WIREPATH = DATAPATH / 'wire.parquet'

## E-transfer aggregation

In [ ]:
#Maaz code here

## Wire transfer aggregation
The aggregate wire transfer score is an average of the wire transfer indicators `w_to_country`, `w_from_country`, `w_external_to_animal`.

In [69]:
wire_df = pd.read_parquet(WIREPATH)
wire_df['w_agg'] = (wire_df['w_to_country'] + wire_df['w_from_country'] + wire_df['w_external_to_animal'])/3
wire_df.sample(3)

,cust_id_sender,cust_id_receiver,name_sender,name_receiver,trxn_value,country_sender,country_receiver,trxn_id,occ_wealth_receiver,occ_animal_receiver,occ_int_receiver,label_receiver,occ_wealth_sender,occ_animal_sender,occ_int_sender,label_sender,w_to_country,w_from_country,w_external_to_animal,w_agg
67027,CUST47410529,CUST56821551,JOHN MILLS,KAREN ALLEN,1626.5,CA,CA,VTTG41908592,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0,0,0,0.0
9599,CUST34075912,EXTERNAL176213,SAMANTHA FERGUSON,ANNA JONES,1544.5,CA,CA,LFUF84927552,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0,0,0,0.0
39203,EXTERNAL959844,CUST35676669,RENEE BATTA,HARRY BROWN,10900.0,IN,CA,FFMZ50593813,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,0,0,0,0.0


## Edge Score Calculation

In [19]:
#Alex to compute total scores for wire transfer and e-transfer inc. his general flags
#Alex to export to a csv with columns: cust_id_sender, cust_id_reciever, score, trxn_type, trxn_id

# Temp

In [2]:
DATAPATH = Path('../data/processed')
EPATH = DATAPATH / 'emt.parquet'
WIREPATH = DATAPATH / 'wire.parquet'

In [3]:
W_OCC_INT = 1/2
W_OCC_ANIMAL_L = 1/4
W_OCC_ANIMAL = 1/4

def compute_score(df, w_occ_int, w_occ_animal_l, w_occ_animal):
    return w_occ_int*df.iloc[2] + w_occ_animal_l*df.iloc[1] + w_occ_animal*df.iloc[0]

edf = pd.read_parquet(EPATH)
sender_cols = ['t_from_animal', 't_from_animal_large', 't_from_int']
receiver_cols = ['t_to_animal', 't_to_animal_large', 't_to_int']
#Compute scores
edf = pd.read_parquet(EPATH)
edf['score_sender'] = edf[sender_cols].apply(compute_score, axis=1, args=(W_OCC_INT, W_OCC_ANIMAL_L, W_OCC_ANIMAL))
edf['score_receiver'] = edf[receiver_cols].apply(compute_score, axis=1, args=(W_OCC_INT, W_OCC_ANIMAL_L, W_OCC_ANIMAL))
#add score_sender and score_receiver and normalize the score to be between 0 and 1
edf['score'] = (edf['score_sender'] + edf['score_receiver'])
edf['score'] = (edf['score'] - edf['score'].min())/(edf['score'].max() - edf['score'].min())
edf.drop(columns=['score_sender', 'score_receiver'], inplace=True)

wdf = pd.read_parquet(WIREPATH)
wdf['score_sender'] = wdf[sender_cols].apply(compute_score, axis=1, args=(W_OCC_INT, W_OCC_ANIMAL_L, W_OCC_ANIMAL))
wdf['score_receiver'] = wdf[receiver_cols].apply(compute_score, axis=1, args=(W_OCC_INT, W_OCC_ANIMAL_L, W_OCC_ANIMAL))
#add score_sender and score_receiver and normalize the score to be between 0 and 1
wdf['score'] = (wdf['score_sender'] + wdf['score_receiver'])
wdf['score'] = (wdf['score'] - wdf['score'].min())/(wdf['score'].max() - wdf['score'].min())
wdf.drop(columns=['score_sender', 'score_receiver'], inplace=True)

In [4]:
outdf = pd.concat([edf,wdf])
outdf.to_parquet(DATAPATH / 'edge_score.parquet', index=False)
outdf.sample(10)

,cust_id_sender,cust_id_receiver,name_sender,name_receiver,trxn_message,emt_value,trxn_id,trxn_type,gender_sender,occupation_sender,...,t_to_animal,t_from_animal,t_to_animal_large,t_from_animal_large,t_to_int,t_from_int,score,trxn_value,country_sender,country_receiver
110518,CUST44476064,CUST34979085,ALICE LAPOINTE,WALTER OLIVER,None,402.5,DAYS67329963,emt,female,Warehouse Worker,...,0,0,0,0,1,0,0.5,NaN,NaN,NaN
143860,EXTERNAL526016,CUST67103093,JULES DROUIN,JOHN FRENCH,None,445.0,TELL92021969,emt,None,None,...,0,0,0,0,0,0,0.0,NaN,NaN,NaN
24550,EXTERNAL196865,CUST54182137,LINDA MARTIN,SUSANNE-AUDREY LAFLAMME,NaN,NaN,KDTB92127479,wire,None,None,...,0,0,0,0,0,0,0.0,3040.0,AU,CA
54905,CUST55193844,CUST16070365,MATHILDE BOIVIN-ÉMOND,CRYSTAL JONES,None,852.0,MTLY49069182,emt,female,Police Officer,...,0,0,0,0,0,0,0.0,NaN,NaN,NaN
375771,CUST58513576,EXTERNAL717542,CAROL DIAZ,VALERIE LARSON,None,472.0,PBYB80856503,emt,female,Construction Contractor,...,0,0,0,0,0,0,0.0,NaN,NaN,NaN
133536,CUST88624000,CUST19744784,IND.BERTRAND DUVAL,DR.BARBARA MILLER,None,345.0,PNSK69958376,emt,other,Student,...,0,0,0,0,1,0,0.5,NaN,NaN,NaN
29742,EXTERNAL983676,CUST41625966,MICHELLE JOHNSON,TERRI PALMER,None,2.5,UPDA73149059,emt,None,None,...,0,0,0,0,0,0,0.0,NaN,NaN,NaN
289302,CUST35974512,EXTERNAL768626,RYAN JAMES PHD,SEAN AYALA,None,385.0,NTXL66145048,emt,male,Photographer,...,0,0,0,0,0,0,0.0,NaN,NaN,NaN
370941,CUST85358613,CUST78599115,ANAYA BISWAS,JOSEPH TURNER,None,85.0,IDXS21001303,emt,male,Hydrologist,...,0,0,0,0,0,0,0.0,NaN,NaN,NaN
303371,EXTERNAL284976,CUST84065578,JUSTIN JENSEN,PIHU SARNA,None,354.0,BKLV90288804,emt,None,None,...,0,0,0,0,0,0,0.0,NaN,NaN,NaN
